In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version:  2.5.0-dev20200629
Eager mode:  True
Hub version:  0.9.0.dev
GPU is available


In [2]:
train = pd.read_csv('train_set.csv')
val = pd.read_csv('val_set.csv')
print(train.sample(5))
print()
print(val.sample(5))

review  rating
37545    The quality of its products less baik.Renda b...       1
2328    Goods wow really, according deh with quality a...       5
100957  Send sensitive kg kg kg kg kg kg kg kg kg. Pac...       4
34508                     Product quality n good price ..       3
54888                      All ori gaes, may fit Aamiin 😇       4

                                                  review  rating
6090                                Good product quality       3
6244    The original product. The product quality is ...       5
14525  You see is gray but the delivery is black (I h...       1
11874                          Excellent product quality       4
8643        Fast delivery Excellent service by seller👍👍👍       5


In [3]:
train_data = train['review'].to_numpy()
train_label = train['rating'].to_numpy()
val_data = val['review'].to_numpy()
val_label = val['rating'].to_numpy()

print(train_data[:5])
print(train_label[:5])

['Package price already received fast delivery murah😁 Thank kasih🙏'
 'Goods have to be in good condition. Sellsr fast response post just a little slow because I want wks ice pack for holding the cheese until melted any ice packs dah hehehe no luck there j & t here we ask today also deliver. Thanks seller. Overall ok'
 'kapaditasnya passable bgt .. thank God'
 ' Awesome awesome quality of goods shipped speed awesome value CP'
 'Good bye but have not I go to hopefully cut no obstacles yaa thanksH']
[4 4 4 5 4]


In [4]:
model_url = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(model_url, output_shape=[20], input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_data[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[-2.2150016e-01, -7.8245920e-01,  4.6349257e-01,  8.4751362e-01,
        -4.7272307e-01, -6.5608144e-01, -2.2218016e-01, -4.2107138e-01,
        -1.2754107e-01, -5.1744974e-01,  1.2923559e-02,  1.1393776e+00,
        -8.0729842e-02,  1.7419104e-01, -5.0030142e-02,  1.6629641e-01,
         1.0719340e+00, -5.5224955e-01,  4.7001633e-01, -5.1118261e-01],
       [ 5.8421504e-01, -8.6143214e-01,  1.0679406e+00,  2.3965011e+00,
        -3.4100821e+00, -2.5437155e+00, -2.1127594e+00,  9.3715662e-01,
         9.2536968e-01, -4.6377125e-01, -1.2880630e+00,  2.0560205e+00,
         3.9489830e-01,  5.6158036e-01, -2.4395983e+00,  4.5074075e-01,
         2.6993909e+00, -8.7518728e-01, -1.2208314e+00, -1.6986959e+00],
       [ 1.8537755e-01, -7.2228426e-01,  1.1669322e+00, -3.8156947e-01,
        -8.1142622e-01, -3.2082266e-01, -4.7196412e-01,  8.1577253e-01,
         1.0289302e+00,  5.6235504e-01, -9.1040117e-01, -1.0004903e+00,
         6.18

In [5]:
model = tf.keras.Sequential([
    hub_layer,
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(5)
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 85        
Total params: 400,441
Trainable params: 400,441
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.compile(optimizer='adam',
              loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [7]:
train_ds = tf.data.Dataset.from_tensor_slices((train_data, train_label))
val_ds = tf.data.Dataset.from_tensor_slices((val_data, val_label))
for data, label in train_ds.take(1):
    print(data)
    print(label)

tf.Tensor(b'Package price already received fast delivery murah\xf0\x9f\x98\x81 Thank kasih\xf0\x9f\x99\x8f', shape=(), dtype=string)
tf.Tensor(4, shape=(), dtype=int64)


In [8]:
train_ds = train_ds.shuffle(10000).batch(64)
val_ds = val_ds.batch(128)

In [9]:
history = model.fit(
    train_ds,
    epochs=40,
    batch_size=128,
    validation_data=val_ds,
    verbose=2
)

Epoch 1/40


InternalError: 2 root error(s) found.
  (0) Internal:  Blas GEMM launch failed : a.shape=(64, 20), b.shape=(20, 16), m=64, n=16, k=20
	 [[node sequential/dense/MatMul (defined at <ipython-input-9-7cf929076523>:6) ]]
  (1) Internal:  Blas GEMM launch failed : a.shape=(64, 20), b.shape=(20, 16), m=64, n=16, k=20
	 [[node sequential/dense/MatMul (defined at <ipython-input-9-7cf929076523>:6) ]]
	 [[gradient_tape/sequential/dense/MatMul/_48]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_884]

Errors may have originated from an input operation.
Input Source operations connected to node sequential/dense/MatMul:
 sequential/keras_layer/StatefulPartitionedCall (defined at C:\Users\Thien\anaconda3\lib\site-packages\tensorflow_hub\keras_layer.py:222)

Input Source operations connected to node sequential/dense/MatMul:
 sequential/keras_layer/StatefulPartitionedCall (defined at C:\Users\Thien\anaconda3\lib\site-packages\tensorflow_hub\keras_layer.py:222)

Functi